<a href="https://colab.research.google.com/github/Mogarthron/Magazynier/blob/master/ZAM_PIANKI_DB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from modele_db import *
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
zam_pianki_xlsx = "Z:/450. PLANISTA - ZAOPATRZENIE/ZAM_PIANKI.xlsx"
pz = pd.read_excel(zam_pianki_xlsx, sheet_name="PZ")
pz_poz = pd.read_excel(zam_pianki_xlsx, sheet_name="PZ_POZ")
aktywne_dos = pd.read_excel(zam_pianki_xlsx, sheet_name="AKTYWNE_DOS")
pw = pd.read_excel(zam_pianki_xlsx, sheet_name="PW")

pz["PZ"] = pz.Numer.apply(lambda x: x[-7:])
pz["DOS"] = pz.KARTA_LIMIT.apply(lambda x: x[4:11])

pz_poz = pz_poz.merge(pz[["PZ", "DOS", "Data", "Obiekt docelowy"]], how="left", on="PZ")

In [3]:
# stmt = (update(ZAM_PIANKI)
#          .where(ZAM_PIANKI.lp.in_([148, 147, 149, 150, 151, 136, 138, 139, 134, 135, 140, 141, 143, 142, 146, 152, 153, 154, 157, 156,  75,  41,  47,  44,  43,  52, 46]))
#         # .where((ZAM_PIANKI.model == "HUDSON"))# & (~ZAM_PIANKI.lp.in_([38,39,48,49,50,51])))
#           .values(status_kompletacja = "1" , nr_pw="24/15"))
# #          # .values(galanteria = "C", siedziska_HR = "C", leniwa = "C"))#, pz = "24/0009"))
# #          # .values(model = "HORIZON"))

# session.execute(stmt)
# session.commit()

In [ ]:
# stmt = (update(ZAM_PIANKI)
#         # .where(ZAM_PIANKI.lp.in_([77]))
#         .where((ZAM_PIANKI.zam1 == "24/0242"))# & (~ZAM_PIANKI.lp.in_([38,39,48,49,50,51])))
#         .values(nr_samochodu = "VITA 7_24"))
#         # .values(potw_dos1="29.01", potw_dos2="29.01>30.01>31.01", nr_kompletacji="1_24"))
#         # .values(galanteria = "C", siedziska_HR = "C", leniwa = "C"))#, pz = "24/0009"))
#         # .values(model = "HORIZON"))

# session.execute(stmt)
# session.commit()

In [ ]:
# delete_stmt = (
#      delete(ZAM_PIANKI)
#      .where(ZAM_PIANKI.lp.in_([28,29])))

# session.execute(delete_stmt)
# session.commit()


In [ ]:
with engine.begin() as conn:
  zam_pianki_db = pd.read_sql(text("select * from ZAM_PIANKI"), conn)
  zamowienia_niedostarczone = pd.read_sql(text("select LP, TYDZIEN, KOD, NR_KOMPLETACJI, OPIS, ILE_ZAMOWIONE, ZNACZNIK_DOSTAWCY, ZAM1, ZAM2, POTW_DATA_DOS_1, POTW_DATA_DOS_2, nr_SAMOCHODU from ZAM_PIANKI where STATUS_KOMPLETACJA is null"), conn)
  zns = pd.read_sql(text("select LP, KOD, NR_KOMPLETACJI, OPIS, ILE_ZAMOWIONE, ZNACZNIK_DOSTAWCY, POTW_DATA_DOS_1, POTW_DATA_DOS_2, DATA_DOSTARCZENIA_1, DATA_DOSTARCZENIA_2, nr_PZ, nr_PW, STATUS_KOMPLETACJA from ZAM_PIANKI where STATUS_KOMPLETACJA is not null and STATUS_KOMPLETACJA <> '1'"), conn)

zns["dostarczone"] = zns.apply(lambda x: x.ZNACZNIK_DOSTAWCY.__len__() == x.STATUS_KOMPLETACJA.__len__(), axis=1)
zamowienia_niespakowane = zns[zns.dostarczone][zns.columns[:-1]]

zpd = zam_pianki_db[["LP", "KOD", "TYDZIEN", "NR_KOMPLETACJI", "OPIS", "ILE_ZAMOWIONE", "ZNACZNIK_DOSTAWCY", "STATUS_KOMPLETACJA"]]
# zam_pianki_db[zam_pianki_db.MODEL == "STONE"][["LP", "MODEL", "TYDZIEN", "OPIS", "ILE_ZAMOWIONE", "ZAM1", "ZAM2", "UWAGI","POTW_DATA_DOS_1","POTW_DATA_DOS_2", "DATA_DOSTARCZENIA_1", "DATA_DOSTARCZENIA_2"]]
#DATY DOSTARCZENIA
# zam_pianki_db[zam_pianki_db.DATA_DOSTARCZENIA_2 == "2024-01-09"][["LP", "nr_PZ", "MODEL", "NR_KOMPLETACJI", "TYDZIEN", "OPIS", "ILE_ZAMOWIONE", "ZAM1", "ZAM2","POTW_DATA_DOS_1","POTW_DATA_DOS_2", "DATA_DOSTARCZENIA_1", "DATA_DOSTARCZENIA_2"]]
#SAMOCHODY
# zam_pianki_db[zam_pianki_db.POTW_DATA_DOS_1 == "29/01"][
#     ["LP", "TYDZIEN", "OPIS", "NR_KOMPLETACJI", "ILE_ZAMOWIONE", "ZAM1", "ZAM2","POTW_DATA_DOS_1", "POTW_DATA_DOS_2", "nr_SAMOCHODU"]]
#PIANKI
# zam_pianki_db[
#     ["LP", "MODEL", "OPIS", "NR_KOMPLETACJI", "GALANTERIA", "SIEDZISKA_HR", "LENIWA"]]
#PW
# _pw = pw[pw.PW == "24/05"].merge(zam_pianki_db[
#      ["LP", "KOD", "ILE_ZAMOWIONE", "DATA_DOSTARCZENIA_1", "DATA_DOSTARCZENIA_2", "ZNACZNIK_DOSTAWCY","STATUS_KOMPLETACJA"]], how="left", on="KOD")
# _pw[(_pw.OPIS.str.contains("REVERSO"))]

def przesuniecie_daty_dostawy(model, nr_kompletacji):
    return zam_pianki_db[(zam_pianki_db.MODEL == model)&(zam_pianki_db.NR_KOMPLETACJI == nr_kompletacji)&(zam_pianki_db.nr_PZ.isna())][["LP","OPIS","ILE_ZAMOWIONE","POTW_DATA_DOS_1","POTW_DATA_DOS_2","ZNACZNIK_DOSTAWCY","STATUS_KOMPLETACJA", "nr_SAMOCHODU"]]


#PZ
def przyjecie_pz(zam,model,nr_kompletacji):
  kol1 = ["LP", "KOD","OPIS","ILE_ZAMOWIONE"]
  if zam == "ZAM1":
    kol2 = ["POTW_DATA_DOS_1"]
  else:
    kol2 = ["POTW_DATA_DOS_2"]

  kol3 = ["ZNACZNIK_DOSTAWCY","STATUS_KOMPLETACJA", "nr_PZ"]

  return zam_pianki_db[(zam_pianki_db.MODEL == model)&(zam_pianki_db.NR_KOMPLETACJI == nr_kompletacji)&(zam_pianki_db.nr_PZ.isna())][kol1+kol2+kol3]


# przesuniecie_daty_dostawy("AMALFI", "2_23")
# zam_pianki_db[(zam_pianki_db.MODEL == "ONYX")&(zam_pianki_db.NR_KOMPLETACJI == "3_23")&(~zam_pianki_db.nr_PZ.isna())][["LP","OPIS","ILE_ZAMOWIONE","POTW_DATA_DOS_1","POTW_DATA_DOS_2","ZNACZNIK_DOSTAWCY","STATUS_KOMPLETACJA", "nr_SAMOCHODU"]]

In [ ]:
# zpd.merge(pw[pw.PW == "24/10"], on="KOD", how="right")
pw_zpd = pw[pw.PW == "24/15"][["PW", "KOD", "OPIS", "ILOSC", "KOMPLETACJA"]].merge(zpd[~zpd.STATUS_KOMPLETACJA.isin(["1",None])][["TYDZIEN", "NR_KOMPLETACJI", "KOD", "LP", "ILE_ZAMOWIONE", "ZNACZNIK_DOSTAWCY", "STATUS_KOMPLETACJA"]], on="KOD", how="left")
# pw_zpd[~pw_zpd.STATUS_KOMPLETACJA.isin(["1",None])]
pw_zpd.dropna().LP.astype(int).values

In [ ]:
nr_dostaw = zam_pianki_db[["LP", "MODEL", "NR_KOMPLETACJI", "OPIS", "DATA_DOSTARCZENIA_1", "DATA_DOSTARCZENIA_2", "nr_SAMOCHODU"]]
nr_dostaw["KOMPLETACJA"] = nr_dostaw.NR_KOMPLETACJI + " " + nr_dostaw.MODEL
nr_dostaw = nr_dostaw[["LP", "KOMPLETACJA", "OPIS", "DATA_DOSTARCZENIA_1", "DATA_DOSTARCZENIA_2", "nr_SAMOCHODU"]]
# nr_dostaw[(nr_dostaw.KOMPLETACJA == "2_23 CALYPSO")]#.LP.values
nr_dostaw

In [ ]:
# stmt_update = (update(ZAM_PIANKI)
#         .where((ZAM_PIANKI.lp.in_([52])))
#         .values(nr_samochodu = "CIECH 8_24,VITA 3_23"))
# session.execute(stmt_update)
# session.commit()

In [ ]:
from sqlalchemy import update, insert, delete
def przyjecie_dostawy(model, nr_kompl, lista_dostarczonych_bryl=None, szybkie_uzupelnienie=None):
  lbp = zam_pianki_db[(zam_pianki_db.MODEL == model)&(zam_pianki_db.NR_KOMPLETACJI == nr_kompl)][["LP", "MODEL", "NR_KOMPLETACJI", "OPIS", "ILE_ZAMOWIONE", "ZNACZNIK_DOSTAWCY", "ZAM1", "ZAM2", "DATA_DOSTARCZENIA_1", "DATA_DOSTARCZENIA_2", "nr_SAMOCHODU", "STATUS_KOMPLETACJA", "nr_PZ",  ]]

  if lista_dostarczonych_bryl == None:
    return lbp
  elif lista_dostarczonych_bryl == True:
    return {i[1].OPIS: {"LP": i[1].LP, "SK": i[1].STATUS_KOMPLETACJA, "DATA_DOS1": i[1].DATA_DOSTARCZENIA_1, "DATA_DOS2": i[1].DATA_DOSTARCZENIA_2, "nr_PZ": i[1].nr_PZ} for i in lbp.iterrows()}
  elif type(lista_dostarczonych_bryl) == dict:
    for k in lista_dostarczonych_bryl.values():
      stmt_update = (update(ZAM_PIANKI)
        .where((ZAM_PIANKI.lp == k["LP"]))
        .values(data_dos1 = k["DATA_DOS1"],
                data_dos2 = k["DATA_DOS2"],
                status_kompletacja = k["SK"],
                nr_pz = k["nr_PZ"]))

      session.execute(stmt_update)
      session.commit()


In [ ]:
zpd[zpd.STATUS_KOMPLETACJA != "1"]

In [ ]:
pz_poz

In [ ]:
data_dos = '2024-03-01'
nr_pz = '24/0037'

przyjecie_dostawy("HORIZON", "2_24")

In [ ]:
with engine.begin() as conn:
  zns = pd.read_sql(text("select LP, NR_KOMPLETACJI, OPIS, ILE_ZAMOWIONE, ZNACZNIK_DOSTAWCY, POTW_DATA_DOS_1, POTW_DATA_DOS_2, DATA_DOSTARCZENIA_1, DATA_DOSTARCZENIA_2, nr_PZ, STATUS_KOMPLETACJA, nr_SAMOCHODU from ZAM_PIANKI where STATUS_KOMPLETACJA is not null and STATUS_KOMPLETACJA <> '1'"), conn)

zns["dostarczone"] = zns.apply(lambda x: x.ZNACZNIK_DOSTAWCY.__len__() == x.STATUS_KOMPLETACJA.__len__(), axis=1)
zns[zns.dostarczone][zns.columns[:-1]]


#29.01>30.01>31.01>12.01 calypso 1_24 pot_data_dos_1

In [ ]:
from sqlalchemy import select

# pod = zam_pianki_db[["LP","OPIS", "NR_KOMPLETACJI", "ILE_ZAMOWIONE", "STATUS_KOMPLETACJA", "ZAM1", "ZAM2", "POTW_DATA_DOS_1", "DATA_DOSTARCZENIA_1", "nr_PZ", "nr_PW"]]
# pod['STATUS_KOMPLETACJA'] = pod.STATUS_KOMPLETACJA.fillna("")
# pod['ZAM1'] = pod.ZAM1.fillna("")
# war = ~(pod.STATUS_KOMPLETACJA.str.contains("P|C"))
# war1 = (pod.ZAM1.str.contains("0007|0036|1486|1517"))
# pod = pod[(pod.STATUS_KOMPLETACJA != "1") & war & war1][["LP", "POTW_DATA_DOS_1"]] #0007,0036,1486,1517


# for i in pod.iterrows():
#   stmt = select(ZAM_PIANKI.lp, ZAM_PIANKI.potw_dos1).where(ZAM_PIANKI.lp == i[1].LP)
#   with engine.connect() as conn:
#     lp, potw_data_dos = list(conn.execute(stmt))[0]
#     print(lp, potw_data_dos+">12.01")

  # stmt_update = (update(ZAM_PIANKI)
  #       .where((ZAM_PIANKI.lp == lp))
  #       .values(potw_dos1 = potw_data_dos+">12.01"))
  # session.execute(stmt_update)
  # session.commit()

In [ ]:
with engine.begin() as conn:
  zam_pianki_db = pd.read_sql(text("select * from ZAM_PIANKI"), conn)

pod = zam_pianki_db[["LP","OPIS", "NR_KOMPLETACJI", "ILE_ZAMOWIONE", "STATUS_KOMPLETACJA", "ZAM1", "ZAM2", "POTW_DATA_DOS_1", "DATA_DOSTARCZENIA_1", "nr_PZ", "nr_PW"]]
pod['STATUS_KOMPLETACJA'] = pod.STATUS_KOMPLETACJA.fillna("")
pod['ZAM1'] = pod.ZAM1.fillna("")
war = ~(pod.STATUS_KOMPLETACJA.str.contains("P|C"))
war1 = (pod.ZAM1.str.contains("0007|0036|1486|1517"))
pod[(pod.STATUS_KOMPLETACJA != "1") & war & war1][["LP", "POTW_DATA_DOS_1"]]

In [ ]:
# stmt = (update(ZAM_PIANKI)
#         .where(ZAM_PIANKI.lp.in_([158, 159, 160, 161, 162, 163, 164, 165, 166, 167]))
#         # .where((ZAM_PIANKI.nr_pz == "24/0022"))# & (~ZAM_PIANKI.lp.in_([38,39,48,49,50,51])))
#         .values(nr_samochodu = "PIANPOL 6_24"))
#         # .values(potw_dos1="29.01", potw_dos2="29.01>30.01>31.01", nr_kompletacji="1_24"))
#         # .values(galanteria = "C", siedziska_HR = "C", leniwa = "C"))#, pz = "24/0009"))
#         # .values(model = "HORIZON"))

# session.execute(stmt)
# session.commit()

In [ ]:
zam_pw = zam_pianki_db
zam_pw[(zam_pw.MODEL == "STONE")&(zam_pw.STATUS_KOMPLETACJA != "1")][["LP", "NR_KOMPLETACJI", "OPIS", "ILE_ZAMOWIONE", "DATA_DOSTARCZENIA_1", "DATA_DOSTARCZENIA_2","STATUS_KOMPLETACJA", "nr_PZ"]]

In [ ]:
pw[pw.PW == "24/03"]

In [ ]:
zamowienia_niedostarczone = zamowienia_niedostarczone.fillna("")
zamowienia_niedostarczone.to_excel("DOS.xlsx")